In [12]:
# imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets


In [5]:
# create Fully Connected Neural Network

class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [22]:
# created a simple CNN

class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(CNN , self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels=8, kernel_size=(3, 3), stride=(1,1) , padding=(1, 1))
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), stride=(1,1) , padding=(1, 1))
        self.fc1 = nn.Linear(16*7*7 , num_classes)
    
    def forward(self , x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)

        return x



In [23]:
model = CNN()
x = torch.randn(64, 1, 28, 28)
print(model(x).shape)

torch.Size([64, 10])


In [24]:
# set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [19]:
# Hyperparameters for fully connected NN.

input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epoch = 1

In [25]:
# Hyperparameters for fully connected CNN.

in_channel = 1
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epoch = 5

In [26]:
# Load Data
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset , batch_size=batch_size , shuffle=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset , batch_size=batch_size , shuffle=True)

In [21]:
# initialize the network
model = NN(input_size , num_classes).to(device)

NameError: name 'NN' is not defined

In [29]:
# for CNN

model = CNN().to(device)

In [30]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters() , lr=learning_rate)

In [36]:
# Train the netwrork
for epoch in range(num_epoch):
    for batch_idx , (data , target) in enumerate(train_loader):
        # get data to cuda if possible
        data = data.to(device)
        target = target.to(device)

        # get to correct shape
        data = data.reshape(data.shape[0] , -1)

        #forward
        scores = model(data)
        loss = criterion(scores , target)

        #backward
        optimizer.zero_grad()
        loss.backward()

        #gradient descent or adam step
        optimizer.step()

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [8, 1, 3, 3], but got 2-dimensional input of size [64, 784] instead

In [37]:
# Train the CNN
for epoch in range(num_epoch):
    for batch_idx , (data , target) in enumerate(train_loader):
        # get data to cuda if possible
        data = data.to(device)
        target = target.to(device)

        #forward
        scores = model(data)
        loss = criterion(scores , target)

        #backward
        optimizer.zero_grad()
        loss.backward()

        #gradient descent or adam step
        optimizer.step()

In [38]:
# check the accuracy 

def check_accuracy(loader , model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x , y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0] , -1)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        
        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples) * 100:.2f} ')
    
    model.train()    


In [39]:
# check the accuracy for CNN

def check_accuracy(loader , model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x , y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        
        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples) * 100:.2f} ')
    
    model.train()    


In [40]:
check_accuracy(train_loader , model)
check_accuracy(test_loader , model)

Got 59083 / 60000 with accuracy 98.47 
Got 9842 / 10000 with accuracy 98.42 
